In [ ]:
!pip install requests
!pip install python-dotenv

In [ ]:
import requests
import base64
import json
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv("/workspaces/ALCS-Sandbox/experimental/.env")

# Configuration for the request
BASE_URL = "https://con1-rest-test.opentext.cloud/dctm-restgenco"
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
OBJECT_ID =os.getenv("OBJECT_ID")
print(f"You entered object ID: {OBJECT_ID}")
CONTENT_URL = f"{BASE_URL}/repositories/genco_acceptance/objects/{OBJECT_ID}/contents/content"

# Encode credentials for Basic Auth
credentials = f"{USERNAME}:{PASSWORD}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()

# Headers
headers = {
    "Accept": "application/vnd.emc.documentum+json",
    "Authorization": f"Basic {encoded_credentials}"
}

# Send request to fetch metadata
response = requests.get(CONTENT_URL, headers=headers)

if response.status_code == 200:
    print("Successfully retrieved metadata")
else:
    print(f"Error retrieving metadata: {response.status_code} - {response.text}")
    exit()

# Extract content from response
data = response.json()

# Extract links
links = data.get('links', [])
# Ensure the folder exists


# Getting the Downloads folder path
downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
os.makedirs(downloads_folder, exist_ok=True)
# Helper to save file
def download_and_save_file(url, filename):
    print(f"Downloading: {url}")
    response = requests.get(url)
    if response.status_code == 200:
        file_path = os.path.join(downloads_folder, filename)
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"{filename} downloaded successfully to {file_path}")
    else:
        print(f"Error downloading {filename}: {response.status_code} - {response.text}")

# Find appropriate link and download
for link in links:
    href = link.get('href', '')
    
    if '.pdf' in href:
        download_and_save_file(href, "document.pdf")
        break

    elif link.get('rel') == 'enclosure' or "mime_type=application/msword" in href:
        download_and_save_file(href, "document.doc")
        break

    elif link.get('rel') == 'enclosure' or "mime_type=image/tiff" in href:
        
        download_and_save_file(href, "document.tif")
        break
